## Weather analysis strategy notebook goals


- Optimise Energy Consumption: Knowing the hours of sunshine and solar irradiation helps teams manage their energy reserves efficiently.
- Plan Strategically: Wind speed and direction data can aid in choosing the best trajectories, ensuring minimum resistance and optimal aerodynamics.
- Enhance Vehicle Design: Understanding temperature fluctuations allows for designing better cooling systems and ensures the solar panels' efficiency isn't compromised.
- Predict Challenges: Past data can help teams anticipate potential hurdles, be it a sudden cloud cover or a gusty wind patch, ensuring they're not caught off-guard.

## Data collection method
By using solcast API, we managed to collect weather and irradiance data from the circuit zolder route coordinates, the data collected in intervals of 5 minutes and using UTC time format.




![API_features](media/API_features.png)

### The chosen features:
1. GHI - "Global Horizontal Irradiance" refers to the total solar radiation received on a horizontal surface, which helps estimate how much energy is available from all incoming solar radiation, including diffuse sunlight, which can affect the car even when it's not directly facing the sun.
2. DNI - "Direct normal irradiance" represents the quantity of radiation received per unit area on a surface perpendicular to the sun.
3. cloud_opacity - Indicates how much sunlight is blocked by clouds.
4. Zenith -
5. Wind Speed (10m) -
6.
7.
8.
9.